# Module 4 Exercises


###  Question 1.
The following Python script scrapes payroll data from the following website: http://kanview.ks.gov/

"KanView is an online solution that brings better visibility, openness, and accountability to Kansas State Government. KanView contains data related to Government spending and income, allowing Kansas taxpayers an inside look at Government financial activity."

The script scrapes the payroll data for the Fort Hays State University (https://www.fhsu.edu/) using Selenium drive along with Beautiful soup parsing library. It stores the scraped results in a Pandas DataFrame. It converts the DataFrame into a corresponding JSON object. It finally writes the JSON object into a json file in your local file system.

Modify this script to scrape the wages data for the University of Kansas (https://ku.edu/)

In [ ]:
%%bash
chmod 777 /tmp
mkdir data
apt-get update --allow-unauthenticated 
apt-get update -y --fix-missing 
pip install selenium
apt-get install chromium-chromedriver -y --fix-missing
pip install joblib
apt-get update --fix-missing

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

mkdir: cannot create directory ‘data’: File exists


In [ ]:
## Headless Browser with BeautifulSoup
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import re                      # for handling regular expressions
import pandas as pd            # using pandas library
from tabulate import tabulate  # tabulate, which takes a list of lists or another tabular data type as the first argument, and outputs a nicely formatted plain-text table


options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = '/usr/bin/chromium-browser'

driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver', options = options)
driver.get("http://kanview.ks.gov/PayRates/PayRates_Agency.aspx")


#After opening the url above, Selenium clicks the specific agency link, in this case FHSU (Fort Hays State University)
python_button = driver.find_element_by_id('MainContent_uxLevel1_Agencies_uxAgencyBtn_86') #FHSU
python_button.click() #click fhsu link


#Selenium hands the page source to Beautiful Soup
soup_level1=BeautifulSoup(driver.page_source, 'lxml')


datalist = [] #empty list
x = 0 #counter

#Beautiful Soup finds all Job Title links on the agency page and the loop begins
for link in soup_level1.find_all('a', id=re.compile("^MainContent_uxLevel2_JobTitles_uxJobTitleBtn_")):
    
    #Selenium visits each Job Title page
    python_button = driver.find_element_by_id('MainContent_uxLevel2_JobTitles_uxJobTitleBtn_' + str(x))
    python_button.click() #click link
    
    #Selenium hands of the source of the specific job page to Beautiful Soup
    soup_level2=BeautifulSoup(driver.page_source, 'lxml')

    #Beautiful Soup grabs the HTML table on the page
    table = soup_level2.find_all('table')[0]
    
    #Giving the HTML table to pandas to put in a dataframe object
    df = pd.read_html(str(table),header=0)
    
    #Store the dataframe in a list
    datalist.append(df[0])
    
    #Ask Selenium to click the back button
    driver.execute_script("window.history.go(-1)") 
    
    #increment the counter variable before starting the loop over
    x += 1
    
    #end loop block
    
#loop has completed

#end the Selenium browser session
driver.quit()


In [ ]:
#combine all pandas dataframes in the list into one big dataframe
result = pd.concat([pd.DataFrame(datalist[i]) for i in range(len(datalist))],ignore_index=True)

#convert the pandas dataframe to JSON
json_records = result.to_json(orient='records')

#pretty print to CLI with tabulate
#converts to an ascii table
print(tabulate(result, headers=["Employee Name","Job Title","Overtime Pay","Total Gross Pay"],tablefmt='psql'))


In [ ]:
#open, writem and download the file to your local file system
from google.colab import files

with open('payroll_data.json', 'w') as f:
  f.write(json_records)

files.download('payroll_data.json')

### Question 2.

Using .decribe() method in Pandas (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html), generate descriptive statistics for Total Gross Pay and Overtime Pay.



In [ ]:
# Answer:

# you may find the following stackoverflow info helpful
# https://stackoverflow.com/questions/32464280/converting-currency-with-to-numbers-in-python-pandas
result['Total Gross Pay'].str.replace('$', '').str.replace(',', '').astype(float).mean()



In [ ]:
result['Overtime Pay'].str.replace('$', '').str.replace(',', '').astype(float).describe()

### Question 3.
Using Pandas, calculate the average Total Gross Pay for Education Program Coordinator.

In [ ]:
# Answer:

epc= result.loc[result['Job Title']== 'Education Program Coordinator']

print('The average total gross pay for an education program coordinator is')
epc['Total Gross Pay'].str.replace('$', '').str.replace(',', '').astype(float).mean()     
